# e) Power & Energy

Calculates power and energy output from the power station for daily river flow data depending on parameters given in a model file.

## Procedure

For each time step (day) in the input river flow data:
1. Determine the required Environmental Flows for each channel
2. Calculate resulting Generation, Canal Flow (and any spill flow)
3. Assign Canal Flow to turbines
4. Calculate headpond and tailwater elevations
5. Calculate headlosses and net head
6. Determine turbine and generator efficiency
7. Calculate power

## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| models.csv                  | | List of models with model paramters identified by Model Name  |
| ngonye_flow_daily_ewrx.csv        | Notebook: d_ewr_setup | Daily flow data annotated with Environmental Flow exceedance values |



## Outputs
| File                       | Description                                 |
|----------------------------|---------------------------------------------|
| [Model Name]_pe_daily.csv  | Daily power output  |


## Libraries and Setup

In [14]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math

## Parameters

In [52]:
#Papermill parameters
#input_file='CC/ngonye_daily_WC2_56' #'ngonye_daily'#'CC/ngonye_daily_WC1_1'
input_file='ngonye_daily'#'CC/ngonye_daily_WC1_1'


In [53]:
input_data='./input_data/'
#output_data='./output_data/2020/'
output_data='./output_data/'

In [54]:
model_name='Base Case'#MW162 'Fixed Tailwater' 'Base Case' Headpond50cm EWRAllCs
dryrun=False #Dont write output files

In [55]:
models=pd.read_csv(input_data + "models.csv").set_index('ModelName')
#models=pd.read_csv(input_data + "2020/models.csv").set_index('ModelName')
model=models.loc[model_name]

out_prefix=model['OutputPrefix']#'base'
if out_prefix!='base':
    out_prefix="scenarios/" + out_prefix

category_set_name=model['EWRCatSet']#'Recommendation 1'
headpond_lift=model['HeadpondLift']#0
plant_capacity=model['Capacity']#180
canal_capacity=plant_capacity/180*1100#1100
fixed_tailwater_level=model['FixedTailwater']#False #970 

#model


## Load Daily Data

Load the daily time series which includes the exceedance values used for calculating daily EWRs.

In [56]:
daily=pd.read_csv(output_data + input_file + '.csv')
daily=daily.set_index(pd.to_datetime(daily['Date'],format="%Y-%m-%d"))
daily=daily.drop('Date',axis=1)
if 'WaterWeek' in daily.columns:
    daily=daily.astype({'WaterWeek': 'int32'})
#daily.info()

In [57]:
daily

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,WaterMonth,WaterDay,WaterWeek,Volume,Flow_difference,Flow_difference_pct,EWRRefExceedance
Date,,,,,,,,,,,,,,,,,
1924-10-01,1924-10-12,100.0000,1.114075,111.407480,0.999,1924,10,1,1924.10,1924,1,1,1,0.009626,0.500000,0.500000,0.50
1924-10-02,1924-10-13,100.0000,1.114075,111.407480,0.999,1924,10,2,1924.10,1924,1,2,1,0.009626,0.000000,0.000000,0.50
1924-10-03,1924-10-14,100.0000,1.114075,111.407480,0.999,1924,10,3,1924.10,1924,1,3,1,0.009626,0.000000,0.000000,0.50
1924-10-04,1924-10-15,100.0000,1.114075,111.407480,0.999,1924,10,4,1924.10,1924,1,4,1,0.009626,0.000000,0.000000,0.50
1924-10-05,1924-10-16,100.0000,1.114075,111.407480,0.999,1924,10,5,1924.10,1924,1,5,1,0.009626,0.000000,0.000000,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-26,2017-10-07,204.0855,0.982689,200.552607,0.955,2017,9,26,2017.09,2016,12,361,52,0.017328,0.000000,0.000000,0.82
2017-09-27,2017-10-08,204.0855,0.982689,200.552607,0.955,2017,9,27,2017.09,2016,12,362,52,0.017328,0.000000,0.000000,0.82
2017-09-28,2017-10-09,204.0855,0.982689,200.552607,0.955,2017,9,28,2017.09,2016,12,363,52,0.017328,0.000000,0.000000,0.82


## EWR Assurance Category Set
Load the available EWR assurance category sets and select one

In [58]:
category_sets=pd.read_csv(input_data + "ewr_section_categories.csv")
#category_sets

In [59]:
category_set=category_sets.loc[:,['Section',category_set_name]]
category_set=category_set.rename(columns={category_set_name:'Category'})
#category_set

Load the EWR Flow Sets which specify the required EWRs by assurance category, channel section and calendar month

In [60]:
ewr_flow_sets=pd.read_csv(input_data + "ewr_flow_sets.csv")
ewr_flow_sets=ewr_flow_sets.set_index(['Section','Category'])
#ewr_flow_sets

## EWR Flow Sets
Select the EWR Flow Set for each channel section according to the Category Set chosen above (eg *Recommendation 1*)

In [61]:
ewrs=category_set.join(ewr_flow_sets,on=['Section','Category']).set_index(['Section','Month'])
ewrs_A=ewrs.loc['A']
ewrs_C=ewrs.loc['C']
ewrs_D=ewrs.loc['D']
ewrs_E=ewrs.loc['E']
ewrs_FG=ewrs.loc['FG']

#ewrs_A


## EWR Flow Bands
Annotate the daily flow series with the EWR Flow Band (Wet, Very Wet, Dry etc) - a lookup against the EWR Exceedance

In [62]:
def flowBandNo(exceed):
    if exceed <= 0.1:
        return 1
    elif  exceed <= 0.4:
        return 2
    elif  exceed <= 0.7:
        return 3
    elif  exceed <= 0.9:
        return 4
    else:
        return 5
    
def flowBandLabel(bandNo):
    if bandNo == 1:
        return 'Wet'
    elif  bandNo == 2:
        return 'Mod_Wet'
    elif  bandNo == 3:
        return 'Normal'
    elif  bandNo ==4:
        return 'Mod_Dry'
    else:
        return 'Dry'    
    
def ewrLookup(section, band, month):
    return ewrs.loc[(section,month)][band]

In [63]:

daily['EWRBandNo']=daily.apply(lambda x: flowBandNo(x['EWRRefExceedance']),axis=1) 
daily['EWRBandLabel']=daily.apply(lambda x: flowBandLabel(x['EWRBandNo']),axis=1) 

## EWR Flows

In [64]:

daily['EWRChannelA']=ewrs_A.lookup(daily['Month'],daily['EWRBandLabel'])
daily['EWRChannelC']=ewrs_C.lookup(daily['Month'],daily['EWRBandLabel'])
daily['EWRChannelD']=ewrs_D.lookup(daily['Month'],daily['EWRBandLabel'])
daily['EWRChannelE']=ewrs_E.lookup(daily['Month'],daily['EWRBandLabel'])
daily['EWRChannelFG']=ewrs_FG.lookup(daily['Month'],daily['EWRBandLabel'])


In [65]:
daily['EWRTotal']=daily['EWRChannelA']+daily['EWRChannelC']+daily['EWRChannelD']+daily['EWRChannelE']+daily['EWRChannelFG']
daily['EWRProportion']=daily['EWRTotal']/daily['Flow']

In [66]:
daily['EWRTotal']=np.where(daily['EWRTotal']>daily['Flow'],daily['Flow'],daily['EWRTotal'])

## Spill and channel flows

In [67]:
daily['FlowCanal']=np.where(daily['Flow']-daily['EWRTotal']>canal_capacity,canal_capacity,daily['Flow']-daily['EWRTotal'])
daily['FlowSpill']=np.where(daily['Flow']-daily['EWRTotal']<=canal_capacity,0,daily['Flow']-daily['EWRTotal']-canal_capacity)

daily['SpillChannelA']=daily['FlowSpill']*(daily['EWRChannelA']/daily['EWRTotal'])
daily['SpillChannelC']=daily['FlowSpill']*(daily['EWRChannelC']/daily['EWRTotal'])
daily['SpillChannelD']=daily['FlowSpill']*(daily['EWRChannelD']/daily['EWRTotal'])
daily['SpillChannelE']=daily['FlowSpill']*(daily['EWRChannelE']/daily['EWRTotal'])
daily['SpillChannelFG']=daily['FlowSpill']*(daily['EWRChannelFG']/daily['EWRTotal'])

daily['FlowChannelA']=daily['SpillChannelA']+daily['EWRChannelA']
daily['FlowChannelC']=daily['SpillChannelC']+daily['EWRChannelC']
daily['FlowChannelD']=daily['SpillChannelD']+daily['EWRChannelD']
daily['FlowChannelE']=daily['SpillChannelE']+daily['EWRChannelE']

daily['TmpA']=np.where(daily['FlowChannelA']>100,daily['FlowChannelA']-100,0)
daily['SpillChannelA']=daily['SpillChannelA']-daily['TmpA']
daily['FlowChannelA']=daily['FlowChannelA']-daily['TmpA']
daily['SpillChannelFG']=daily['SpillChannelFG']+daily['TmpA']

daily['TmpC']=np.where(daily['FlowChannelC']>18,daily['FlowChannelC']-18,0)
daily['SpillChannelC']=daily['SpillChannelC']-daily['TmpC']
daily['FlowChannelC']=daily['FlowChannelC']-daily['TmpC']
daily['SpillChannelFG']=daily['SpillChannelFG']+daily['TmpC']

daily['TmpD']=np.where(daily['FlowChannelD']>25,daily['FlowChannelD']-25,0)
daily['SpillChannelD']=daily['SpillChannelD']-daily['TmpD']
daily['FlowChannelD']=daily['FlowChannelD']-daily['TmpD']
daily['SpillChannelFG']=daily['SpillChannelFG']+daily['TmpD']

daily['TmpE']=np.where(daily['FlowChannelE']>80,daily['FlowChannelE']-80,0)
daily['SpillChannelE']=daily['SpillChannelE']-daily['TmpE']
daily['FlowChannelE']=daily['FlowChannelE']-daily['TmpE']
daily['SpillChannelFG']=daily['SpillChannelFG']+daily['TmpE']

daily['FlowChannelFG']=daily['SpillChannelFG']+daily['EWRChannelFG']
daily=daily.drop(['TmpA','TmpC','TmpD','TmpE'],axis=1)

daily['FlowLeftChannel']=daily['FlowChannelA']+daily['FlowChannelC']+daily['FlowChannelD']+daily['FlowCanal']


In [68]:

daily['Turbines']=daily.apply(lambda x: math.ceil(x['FlowCanal'] / (canal_capacity/4)),axis=1)
daily.loc[daily['FlowCanal']<=plant_capacity/180*50,'Turbines']=0

In [69]:
daily['FlowTurbine1']=np.where(daily['Turbines']>=1,daily['FlowCanal']/daily['Turbines'],np.nan)
daily['FlowTurbine2']=np.where(daily['Turbines']>=2,daily['FlowCanal']/daily['Turbines'],np.nan)
daily['FlowTurbine3']=np.where(daily['Turbines']>=3,daily['FlowCanal']/daily['Turbines'],np.nan)
daily['FlowTurbine4']=np.where(daily['Turbines']==4,daily['FlowCanal']/daily['Turbines'],np.nan)

## Levels, headloss and head


\begin{equation}
  Tailwater Level=\begin{cases}
    -2.6828*log_{10}(Q)^2+11.265*log_{10}(Q)+952.68, & \text{if $log_{10}(Q) > 3.8$}.\\
    -21.429*log_{10}(Q)^2+182.03*log_{10}(Q)+597.15, & \text{if $2.3 < log_{10}(Q) < 3.8$}.\\
    6.5299*log_{10}(Q)^2-30.155*log_{10}(Q)+999.52, & \text{if $log_{10}(Q) < 2.3$}.
  \end{cases}
\end{equation}

\begin{equation}
  Headpond Level=\begin{cases}
    990.0, & \text{if $Q<5000$}.\\
    0.000081307*Q+989.6, & \text{if $Q<7500$}.\\
    0.000240*Q+988.4, & \text{otherwise}.
  \end{cases}
\end{equation}

\begin{equation}
Headloss Powerhouse=0.00000702Q^2 + 0.00002226Q - 0.00207422
\end{equation}

In [70]:
def headpondLevel(flow):
    if flow<5000:
        return 990.0 + headpond_lift
    elif flow<7500:
        return 0.000081307*flow+989.6 + headpond_lift
    else:
        return 0.000240*flow+988.4 + headpond_lift

In [71]:
def tailwaterLevel(flow):
    if fixed_tailwater_level>0:
        return fixed_tailwater_level
    else:
        log_flow=math.log10(flow)
        if log_flow>=3.8:
            return -21.429*log_flow**2+182.03*log_flow+597.15   
        elif log_flow>=2.3:
            return 6.5299*log_flow**2-30.155*log_flow+999.52   
        else:
            return -2.6828*log_flow**2+11.265*log_flow+952.68

In [72]:
def turbineHeadloss(flow):
    if np.isnan(flow):
        return np.nan
    elif flow==0:
        return 0
    else:
        return 0.00000702*flow**2+0.00002226*flow-0.00207422

In [73]:
daily['LevelTailwater']=daily['Flow'].apply(tailwaterLevel)
daily['LevelHeadpond']=daily['Flow'].apply(headpondLevel)

In [74]:
daily.loc[daily['FlowCanal']==-4.614652935489659]['EWRProportion']

Series([], Name: EWRProportion, dtype: float64)

In [75]:
headloss_leftchannel=pd.read_csv(input_data + "headloss_leftchannel.csv")
fHeadlossLeftChannel = interpolate.interp1d(headloss_leftchannel['FlowLeftChannel'], headloss_leftchannel['HeadlossLeftChannel'])
daily['HeadlossLeftChannel']=daily['FlowLeftChannel'].apply(fHeadlossLeftChannel)

In [76]:
headloss_canal=pd.read_csv(input_data + "headloss_canal.csv")
fHeadlossCanal = interpolate.interp1d(headloss_canal['FlowCanal'], headloss_canal['HeadlossCanal'])

daily['HeadlossCanal']=daily['FlowCanal'].apply(fHeadlossCanal)
daily['LevelForebay']=daily['LevelHeadpond']-daily['HeadlossLeftChannel']-daily['HeadlossCanal']

In [77]:
daily['HeadlossTurbine1']=daily['FlowTurbine1'].apply(turbineHeadloss)
daily['HeadlossTurbine2']=daily['FlowTurbine2'].apply(turbineHeadloss)
daily['HeadlossTurbine3']=daily['FlowTurbine3'].apply(turbineHeadloss)
daily['HeadlossTurbine4']=daily['FlowTurbine4'].apply(turbineHeadloss)

In [78]:
daily['HeadTurbine1']=np.where(daily['FlowTurbine1']>0,daily['LevelForebay']-daily['LevelTailwater']-daily['HeadlossTurbine1'],np.nan)
daily['HeadTurbine2']=np.where(daily['FlowTurbine2']>0,daily['LevelForebay']-daily['LevelTailwater']-daily['HeadlossTurbine2'],np.nan)
daily['HeadTurbine3']=np.where(daily['FlowTurbine3']>0,daily['LevelForebay']-daily['LevelTailwater']-daily['HeadlossTurbine3'],np.nan)
daily['HeadTurbine4']=np.where(daily['FlowTurbine4']>0,daily['LevelForebay']-daily['LevelTailwater']-daily['HeadlossTurbine4'],np.nan)
daily.loc[daily['HeadTurbine1']<=10,'Turbines']=0

## Turbine and generator efficiency


\begin{equation*}
  Generator Efficiency=\begin{cases}
    98\%, & \text{if $LoadFactor>0.85$}.\\
    97\%, & \text{if $0.45>LoadFactor<0.85$}.\\
    96\%, & \text{if $LoadFactor<0.45$}.
  \end{cases}
\end{equation*}


In [79]:
def generatorEff(load_fact):
    if np.isnan(load_fact):
        return np.nan
    
    if load_fact==0:
        return 0
    elif load_fact<=0.45:
        return 0.96
    elif load_fact<=0.85:
        return 0.97
    else:
        return 0.98

In [80]:
def turbineEfficiency(q_actual,h):
    
    #Scale flow to installed capacity
    q=q_actual*(180/plant_capacity)
    
    if np.isnan(h) or np.isnan(q): return np.nan
    
    if h<=10: return 0
    
    if q==0: return 0
    elif q<53.5: return 0.77
    elif q<55: return 0.78
    elif q<58.5: return 0.79
    elif q<62: return 0.80
    elif q<64: return 0.81
    elif q<66.5: return 0.82
    elif q<70: return 0.83
    elif q<72: return 0.84
    elif q<74.5: return 0.85
    elif q<82.5: return 0.86
    elif q<91.5: return 0.87
    elif q<100.5: return 0.88
    elif q<109.72: return 0.89
    elif q<121.77: return 0.90
    elif q<146.8: return 0.91
    elif q<169.545: return 0.92
    elif q<173.1:
        if h<24.65: return 0.92
        else: return 0.93
    elif q<199.8: return 0.93
    elif q<211.2:
        if h<23: return 0.93
        else: return 0.94
    elif q<247.05:
        return 0.94
    elif q<250:
        if h<24.16: return 0.94
        else: return 0.95
    elif q<260:
        if h<23.6: return 0.94
        else: return 0.95
    elif q<275:
        if h<22.5: return 0.94
        else: return 0.95
    else:
        return -0.0015*h**2 + 0.0565*h + 0.4

In [81]:
daily['EffTurbine1']=daily.apply(lambda x: turbineEfficiency(x['FlowTurbine1'],x['HeadTurbine1']),axis=1)
daily['EffTurbine2']=daily.apply(lambda x: turbineEfficiency(x['FlowTurbine2'],x['HeadTurbine2']),axis=1)
daily['EffTurbine3']=daily.apply(lambda x: turbineEfficiency(x['FlowTurbine3'],x['HeadTurbine3']),axis=1)
daily['EffTurbine4']=daily.apply(lambda x: turbineEfficiency(x['FlowTurbine4'],x['HeadTurbine4']),axis=1)

In [82]:
daily['LoadFactorTurbine1']=daily['FlowTurbine1']/250*(plant_capacity/180)
daily['LoadFactorTurbine2']=daily['FlowTurbine2']/250*(plant_capacity/180)
daily['LoadFactorTurbine3']=daily['FlowTurbine3']/250*(plant_capacity/180)
daily['LoadFactorTurbine4']=daily['FlowTurbine4']/250*(plant_capacity/180)

In [83]:
daily['EffGen1']=daily['LoadFactorTurbine1'].apply(generatorEff)
daily['EffGen2']=daily['LoadFactorTurbine2'].apply(generatorEff)
daily['EffGen3']=daily['LoadFactorTurbine3'].apply(generatorEff)
daily['EffGen4']=daily['LoadFactorTurbine4'].apply(generatorEff)

## Power and energy

In [84]:
daily['PowerTurbine1']=daily['FlowTurbine1']*daily['HeadTurbine1']*daily['EffTurbine1']*daily['EffGen1']*9.81/1000
daily['PowerTurbine2']=daily['FlowTurbine2']*daily['HeadTurbine2']*daily['EffTurbine2']*daily['EffGen2']*9.81/1000
daily['PowerTurbine3']=daily['FlowTurbine3']*daily['HeadTurbine3']*daily['EffTurbine3']*daily['EffGen3']*9.81/1000
daily['PowerTurbine4']=daily['FlowTurbine4']*daily['HeadTurbine4']*daily['EffTurbine4']*daily['EffGen4']*9.81/1000
daily['Power']=np.nan_to_num(daily['PowerTurbine1'])+np.nan_to_num(daily['PowerTurbine2'])+np.nan_to_num(daily['PowerTurbine3'])+np.nan_to_num(daily['PowerTurbine4'])
daily['Energy']=daily['Power']*24
daily['PerfMwPerCumec']=daily['Power']/daily['Flow']

In [95]:
daily.loc[daily["PowerTurbine1"]==0]
daily['LowHeadShutoff']=np.where(daily["PowerTurbine1"]==0,1,0)
daily['LowFlowShutoff']=np.where(daily["EffGen1"].isnull(),1,0)
#daily['LowHeadShutoff']=np.where(daily["PowerTurbine1"]==0,1,0)
#daily.loc[daily["PowerTurbine1"].isnull()]['LowHeadShutoff']=1

In [99]:
if dryrun==False:
    daily.to_csv(output_data+input_file + '_' + out_prefix+'_pe.csv')

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,...,EffGen4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec,LowHeadShutoff,LowFlowShutoff
Date,,,,,,,,,,,,,,,,,,,,,
1924-10-01,1924-10-12,100.0,1.114075,111.40748,0.999,1924,10,1,1924.1,1924,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,1
1924-10-02,1924-10-13,100.0,1.114075,111.40748,0.999,1924,10,2,1924.1,1924,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,1
1924-10-03,1924-10-14,100.0,1.114075,111.40748,0.999,1924,10,3,1924.1,1924,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,1
1924-10-04,1924-10-15,100.0,1.114075,111.40748,0.999,1924,10,4,1924.1,1924,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,1
1924-10-05,1924-10-16,100.0,1.114075,111.40748,0.999,1924,10,5,1924.1,1924,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,1
1924-10-06,1924-10-17,100.0,1.114075,111.40748,0.999,1924,10,6,1924.1,1924,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,1
1924-10-07,1924-10-18,100.0,1.114075,111.40748,0.999,1924,10,7,1924.1,1924,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,1
1924-10-08,1924-10-19,100.0,1.114075,111.40748,0.999,1924,10,8,1924.1,1924,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,1
1924-10-09,1924-10-20,100.0,1.114075,111.40748,0.999,1924,10,9,1924.1,1924,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,1
